Simple script to fetch all the latest measurement probe data in Africa, create seperate csv files per country, and cocatenate into a single csv file.

***This notebook, and csv files must be in the same directory***

***Delete all the csv files in the directory if you need to run this script again***

In [4]:
import requests
import json
from pandas.io.json import json_normalize 
import time
from datetime import date
import pandas as pd
import _thread
import matplotlib.pyplot as plt
import matplotlib.dates
import matplotlib as mpl
from datetime import datetime
import numpy as np
import csv
import os
import glob


os.chdir('/home/tunde/GIT_PROJECTS/RIPE_ANALYSIS/African_Probes_Status')

%matplotlib inline

#set ggplot style
plt.style.use('default')
plt.rcParams.update({'font.size': 14})

ATLAS_PROBES_URI = "https://stat.ripe.net/data/atlas-probes/data.json?resource="
ATLAS_DEPLOYMENT_URI = "https://stat.ripe.net/data/atlas-probe-deployment/data.json?resource="

ccs = [
'EG', 'BJ', 'CI', 'CV', 'GH', 'GM', 'GN', 'GW', 'AO', 'CF', 'CG', 'CM', 'GA', 'GQ', 'TD', 'BI', 'DJ', 'ER', 'ET', 'KM', 'BW', 'MA', 'SD', 'TN', 'LR', 'ML', 'MR', 'NE', 'NG', 'SL', 'SN', 'TG', 'ST', 'KE', 'MG', 'MU', 'MW', 'MZ', 'RE', 'RW', 'SC', 'SO', 'UG', 'LS', 'NA', 'SZ', 'ZA', 'DZ', 'EH', 'LY', 'BF', 'SH', 'CD', 'TZ', 'YT', 'ZM', 'ZW']

## Function to get probe status per country and create a seperate csv file per country in the list ***ccs*** above

In [5]:
def getDeployment(cc):
    
    test_url = ATLAS_DEPLOYMENT_URI + cc
    
    df = pd.DataFrame()
    
    try:
        r = requests.get(test_url)
    except requests.exceptions.RequestException as e:
        return "Request FAILED"
    
    try:
        data = json.loads(r.text)['data']['deployments'][0]['deployment']
        df = json_normalize(data)
        df['cc'] = cc
    except KeyError as e:
        pass
    
    return df

## Loop through all the created csv files in the present directory the notebook is

In [7]:
for cc in ccs:
        os.chdir('/home/tunde/GIT_PROJECTS/RIPE_ANALYSIS/African_Probes_Status/csv_data_files')
        print(cc)
        a = getDeployment(cc)
        a.to_csv('getDep__'+ cc +'__latest_deployments.csv')

EG
BJ
CI
CV
GH
GM
GN
GW
AO
CF
CG
CM
GA
GQ
TD
BI
DJ
ER
ET
KM
BW
MA
SD
TN
LR
ML
MR
NE
NG
SL
SN
TG
ST
KE
MG
MU
MW
MZ
RE
RW
SC
SO
UG
LS
NA
SZ
ZA
DZ
EH
LY
BF
SH
CD
TZ
YT
ZM
ZW


## Merge the csv files created above


In [8]:
extension = 'csv'

os.chdir('/home/tunde/GIT_PROJECTS/RIPE_ANALYSIS/African_Probes_Status/csv_data_files')

all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])

#export to csv
combined_csv.to_csv( "2020_africa_probes_deployment.csv", index=False, encoding='utf-8-sig')

print('csv files successfully merged')

csv files successfully merged
